In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [58]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=5,
    lora_alpha=15,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key', 'value', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 962,308 || all params: 125,611,016 || trainable%: 0.7661


In [59]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [60]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.347000,1.187454,0.878125
200,0.569500,0.348814,0.884375
300,0.355600,0.345436,0.895312
400,0.286100,0.344021,0.898438
500,0.309200,0.326809,0.904687
600,0.290300,0.318602,0.900000
700,0.296800,0.319222,0.906250
800,0.312800,0.308457,0.903125
900,0.294400,0.307025,0.906250
1000,0.280500,0.302675,0.906250


In [62]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-1200'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.19it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-1200.csv


In [61]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3470,1.187454,0.430625,0.878125,0.000048,0.013405,0.159546,1.134360,-0.447500,0.490391
1,200,0.5695,0.348814,0.867857,0.884375,0.000045,0.026810,0.220686,1.632677,-0.016518,0.981323
2,300,0.3556,0.345436,0.885714,0.895312,0.000043,0.040214,0.010164,1.029424,-0.009598,0.989279
3,400,0.2861,0.344021,0.906250,0.898438,0.000040,0.053619,-0.057921,0.831636,0.007812,1.008696
4,500,0.3092,0.326809,0.897321,0.904687,0.000037,0.067024,-0.017609,0.946119,-0.007366,0.991858
5,600,0.2903,0.318602,0.904464,0.900000,0.000035,0.080429,-0.028302,0.911168,0.004464,1.004960
6,700,0.2968,0.319222,0.900000,0.906250,0.000033,0.093834,-0.022422,0.929762,-0.006250,0.993103
7,800,0.3128,0.308457,0.899554,0.903125,0.000030,0.107239,0.004343,1.014078,-0.003571,0.996045
8,900,0.2944,0.307025,0.901786,0.906250,0.000028,0.120643,-0.012625,0.958879,-0.004464,0.995074
9,1000,0.2805,0.302675,0.904464,0.906250,0.000025,0.134048,-0.022175,0.926736,-0.001786,0.998030
